In [148]:
import geopandas as gp
import os
import pandas as pd
import unicodedata
import folium
import branca

In [149]:
gf = gp.read_file(filename=os.path.abspath('geojs-100-mun.json'))

In [150]:
accidents = pd.read_csv('accidents.csv')

In [151]:
# Adiciona a coluna name_normal que contem os municipios sem acentos
gf['name_normal'] = gf.apply(lambda row: unicodedata.normalize('NFD', row['name']).encode('ansi', 'ignore').decode(), axis=1)

In [152]:
gf['pais'] = "Brasil"

In [153]:
gb = gp.read_file(filename='Brasil.json')

gb['pais'] = 'Brasil'

In [154]:
acc_calc = pd.DataFrame(accidents.ocorrencia_uf.unique(), columns=['UF'])

In [155]:
total = [[x[0], accidents[accidents['ocorrencia_uf'] == x[0]].ocorrencia_uf.count()] for x in acc_calc.values if x[0] != '***']

In [156]:
dt=pd.DataFrame(total, columns=['UF', 'TOTAL'])

In [157]:
dt['PERCENTUAL'] = dt.apply(lambda row : (row['TOTAL']/ dt.TOTAL.sum()) * 100,axis=1)

In [158]:
gdf = gp.read_file('Brasil.json')

In [159]:
def map_build():
    map = folium.Map(location=[-16,-49], zoom_start=4)
#     folium.GeoJson(gb.to_json()).add_to(map)
    return map

In [160]:
m = map_build()
m.choropleth(geo_data='Brasil.json',
            data= dt,
            columns=['UF', 'PERCENTUAL'],
            key_on='feature.properties.UF',
            fill_color='YlOrRd',
            highlight=False
            )

m.get_root().header.add_child(branca.element.Element('<title>Acidentes da aviação</title>'))

In [161]:
dt[dt.UF == x[1].UF].PERCENTUAL.values[0] * 100

92.1435499515034

In [162]:
for x in gdf.iterrows():
    estado = gdf[gdf.UF == dt[dt.UF == x[1].UF].UF.values[0]].ESTADO.values[0]
    perc = str(int(dt[dt.UF == x[1].UF].PERCENTUAL.values[0]))
    text = '<b> ' + estado + '<br>' + perc + '%'
    
    folium.Marker(location=[x[1].geometry.centroid.y,x[1].geometry.centroid.x], popup=text, tooltip=estado).add_to(m)

In [163]:
m

In [164]:
m.save('Mapeamento de Acidentes.html')

In [166]:
m._to_png()

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\ALEXAN~1\\AppData\\Local\\Temp\\tmpoxrbqgic.html'